## COMETH vs. UE Angle Extraction

**Author:**  Shafagh Keyvanian [shkey@seas.upenn.edu]  
**Date**: *August 2024*

### For each Subject
Converts Motive export .csv to Upper-Extremitity Euler angles

Input: OptiTrack data: Motive exported .csv file
- Skeleton: Conventional Upper (27 Markers) - 43 Bones
- Gaps manually filled with "Linear Interpolation"
- Export: Bones- position & quaternion, Markers- position

Output: npz file with fields: time, bones, euler_angles
- Saves .npz file to folder: edited_exports\date
- File name: 1st-3rd word of take name, folder: Subject
- Bones fields: rows=frames, col=(quat(x y z w) pos[x y z])

In [ ]:
#%% import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
from tf.transformations import quaternion_matrix, euler_from_matrix, euler_matrix

import nimblephysics as nimble
gui = nimble.NimbleGUI()
gui.serve(8080)

In [ ]:
#%% Define chains and DOFs
chain_R = ['Skeleton', 'Ab', 'Chest','RShoulder','RUArm','RFArm','RHand']
chain_L = ['Skeleton', 'Ab', 'Chest','LShoulder','LUArm','LFArm','LHand']
chain_H = ['Skeleton', 'Ab', 'Chest','Neck', 'Head']

HUB_dof_R = ['pelvis_tilt', 'pelvis_list', 'pelvis_rotation',   'Ab_bending', 'Ab_rotation', 'Ab_twist',    'Chest_bending', 'Chest_rotation', 'Chest_twist', \
             'RClavicle Protraction', 'RClavicle Elevation','RClavicle Axial Rotation',     'Shoulder Abduction', 'Shoulder Rotation', 'Shoulder Extension', \
             'Elbow Flexion', 'Elbow Supination', 'Elbow_0',    'Wrist Deviation', 'Wrist Supination', 'Wrist Flexion']

HUB_dof_L = ['pelvis_tilt', 'pelvis_list', 'pelvis_rotation',   'Ab_bending', 'Ab_rotation', 'Ab_twist',    'Chest_bending', 'Chest_rotation', 'Chest_twist', \
             'LClavicle Protraction', 'LClavicle Elevation', 'LClavicle Axial Rotation',    'Shoulder Abduction', 'Shoulder Rotation', 'Shoulder Extension', \
             'Elbow Flexion', 'Elbow Supination', 'Elbow_0',    'Wrist Deviation', 'Wrist Supination', 'Wrist Flexion']
             
HUB_dof_H = ['pelvis_tilt', 'pelvis_list', 'pelvis_rotation',   'Ab_bending', 'Ab_rotation', 'Ab_twist',    'Chest_bending', 'Chest_rotation', 'Chest_twist', \
             'Neck Bending', 'Neck Rotation', 'Neck Twist',     'Head Flexion', 'Head Rotation', 'Head Twist']

cometh_chain_R = ['ground_pelvis', 'lumbar', 'thoracic_spine','scapulothoracic_r','GlenoHumeral_r','elbow_r','wrist_r']
cometh_chain_L = ['ground_pelvis', 'lumbar', 'thoracic_spine','scapulothoracic_l','GlenoHumeral_l','elbow_l','wrist_l']
cometh_chain_H = ['ground_pelvis', 'lumbar', 'thoracic_spine','neck','neck']

cometh_dof_R = ['pelvis_tilt', 'pelvis_list', 'pelvis_rotation','lumbar_bending', 'lumbar_extension', 'lumbar_twist', 'thorax_bending', 'thorax_extension', 'thorax_twist', \
                'scapula_abduction_r', 'scapula_elevation_r', 'scapula_upward_rot_r', 'shoulder_r_z', 'shoulder_r_x', 'shoulder_r_y', \
                'elbow_flexion_r', 'pro_sup_r', 'elbow_0_r', 'wrist_flexion_r', 'wrist_deviation_r', 'wrist_0_r']

cometh_dof_L = ['pelvis_tilt', 'pelvis_list', 'pelvis_rotation','lumbar_bending', 'lumbar_extension', 'lumbar_twist', 'thorax_bending', 'thorax_extension', 'thorax_twist', \
                'scapula_abduction_l', 'scapula_elevation_l', 'scapula_upward_rot_l', 'shoulder_l_x', 'shoulder_l_y', 'shoulder_l_z', \
                'elbow_flexion_l', 'pro_sup_l', 'elbow_0_l', 'wrist_flexion_l', 'wrist_deviation_l', 'wrist_0_l']
                    
cometh_dof_H = ['pelvis_tilt', 'pelvis_list', 'pelvis_rotation','lumbar_bending', 'lumbar_extension', 'lumbar_twist', 'thorax_bending', 'thorax_extension', 'thorax_twist', \
                'head_bending', 'head_extension', 'head_twist', 'head_0_x', 'head_0_y', 'head_0_z']


#%% Load COMETH skeleton
bsm_dir = '/home/shkey/catkins/human_model_ws/src/COMETH/COMETH/bsm_shkey.osim'
geom_dir = '/home/shkey/catkins/human_model_ws/src/COMETH/COMETH/Geometry/'
skeleton = nimble.biomechanics.OpenSimParser.parseOsim(bsm_dir, geom_dir).skeleton
p0 = skeleton.getPositions()

angles = []
for i in range(skeleton.getNumJoints()):
    j = skeleton.getJoint(i)
    for k in range(j.getNumDofs()):
        angles.append(j.getDofName(k))
        
p = p0.copy()
for i in range(len(angles)):
    p[i] = 0

#%% Load HUB skeleton
csvfile = '../data/raw_exports/subject4_Rh_clinical.csv' 
raw_data = pd.read_csv(csvfile, header=[3, 5, 6], index_col=0)
raw_data.rename(columns=lambda x: x.replace('Skeleton:', ''), level=0, inplace=True)
time = pd.read_csv(csvfile, header=6, usecols=[1]).values
skl_R, skl_L, skl_H = raw_data[chain_R].copy(), raw_data[chain_L].copy(), raw_data[chain_H].copy()

#%% Get HUB skeleton dimensions
t = 5000
# Offset angle between strnm_dir_R/L and strnm_dir_H
strnm_dir_R = skl_R[('RShoulder', 'Position')].iloc[t] - skl_R[('Chest', 'Position')].iloc[t]
strnm_dir_L = skl_L[('LShoulder', 'Position')].iloc[t] - skl_L[('Chest', 'Position')].iloc[t]
strnm_dir_H = skl_H[('Neck', 'Position')].iloc[t] - skl_H[('Chest', 'Position')].iloc[t]
strnm_ang_R = np.arccos(np.dot(strnm_dir_R, strnm_dir_H)/(np.linalg.norm(strnm_dir_R)*np.linalg.norm(strnm_dir_H)))
strnm_ang_L = np.arccos(np.dot(strnm_dir_L, strnm_dir_H)/(np.linalg.norm(strnm_dir_L)*np.linalg.norm(strnm_dir_H)))
# print(skl_H[('Chest', 'Position')].iloc[t]-skl_H[('Skeleton', 'Position')].iloc[t])

def get_positions(skl, chain):  
    positions = []
    for j in range(len(chain)):
        pos = skl[(chain[j], 'Position')].iloc[t]
        positions.append(np.array(pos))
    return positions - positions[0]

def get_bone_length(skl, chain):  
    positions = []
    for j in range(len(chain)):
        pos = skl[(chain[j], 'Position')].iloc[t]
        positions.append(np.array(pos))
    positions -= positions[0]
    bone_length = np.linalg.norm(np.diff(positions, axis=0), axis=1)    
    return bone_length

In [ ]:
# Initialize COMETH and HUB Angles
angles_HUB_base = [[0, 0, 0],  # 'pelvis_tilt', 'pelvis_list', 'pelvis_rotation'
                   [0, 0, 0],  # 'Ab_bending', 'Ab_extension', 'Ab_twist'
                   [0, 0, 0]]  # 'Chest_bending', 'Chest_extension', 'Chest_twist'
                   
angles_HUB_R = np.vstack([angles_HUB_base, 
               [[0, 0, 0],      # RClavicle (Protraction, Elevation, Rotation) --- (abduction, elevation, upward_rot)
                [0, 90, 0],     # 'Shoulder Abduction', 'Shoulder Rotation', 'Shoulder Extension'
                [30, 0, 0],     # 'Elbow Flexion', 'Elbow_0', 'Elbow Supination'
                [0, 0, 0]]])    # 'Wrist Flexion', 'Wrist_0', 'Wrist Deviation'

angles_HUB_L = np.vstack([angles_HUB_base,
                [[0, 0, 0],     # LClavicle (Protraction, Elevation, Rotation) --- (abduction, elevation, upward_rot)
                 [0, 90, 0],    # 'Shoulder Abduction', 'Shoulder Rotation', 'Shoulder Extension'
                 [30, 0, 0],    # 'Elbow Flexion', 'Elbow_0', 'Elbow Supination'
                 [0, 0, 0]]])   # 'Wrist Flexion', 'Wrist_0', 'Wrist Deviation'

angles_HUB_H = np.vstack([angles_HUB_base,
                [[0, 0, 0],      # 'Neck Bending', 'Neck Rotation', 'Neck Twist'    
                 [0, 0, 0]]])    # 'Head Flexion', 'Head Rotation', 'Head Twist'

angles_HUB_R = np.array(angles_HUB_R) * np.pi/180
angles_HUB_L = np.array(angles_HUB_L) * np.pi/180
angles_HUB_H = np.array(angles_HUB_H) * np.pi/180

p[0:3]   = angles_HUB_H[0]                # pelvis_tilt, pelvis_list, pelvis_rotation
# p[3:6]   = [0, 0, 0]                    # pelvis_tx, pelvis_ty, pelvis_tz
p[20:23] = angles_HUB_H[1]                # lumbar_bedning, lumbar_extension, lumbar_twist
p[23:26] = angles_HUB_H[2]                # thorax_bending, thorax_extension, thorax_twist
p[26:29] = angles_HUB_H[3]                # head_bending, head_extension, head_twist

p[29:32] = angles_HUB_R[3]                # scapulothoracic_r (abduction, elevation, upward_rot)
p[32:35] = angles_HUB_R[4]                # GlenoHumeral_r (shoulder_r_z, shoulder_r_x, shoulder_r_y)
p[35:37] = angles_HUB_R[5,[0,2]]          # elbow_r (elbow_flexion, pro_sup)
p[37:39] = angles_HUB_R[6,[0,2]]          # wrist_r (wrist_flexion, wrist_deviation)

p[39:42] = angles_HUB_L[3]                # scapulothoracic_l (abduction, elevation, upward_rot)
p[42:45] = angles_HUB_L[4]                # GlenoHumeral_l (shoulder_l_x, shoulder_l_y, shoulder_l_z)
p[45:47] = angles_HUB_L[5,[0,2]]          # elbow_l (elbow_flexion, pro_sup)
p[47:49] = angles_HUB_L[6,[0,2]]          # wrist_l (wrist_flexion, wrist_deviation)

skeleton.setPositions(p)
gui.nativeAPI().renderSkeleton(skeleton)

joint_positions = skeleton.getJointWorldPositionsMap()
for i in range(len(cometh_chain_H)):
    print(i, cometh_chain_H[i], joint_positions[cometh_chain_H[i]])
    gui.nativeAPI().createSphere(key=cometh_chain_H[i], radii=np.array([0.01,0.01,0.01]), pos=joint_positions[cometh_chain_H[i]], color=np.array([0,255,0,0.8]))
print()
for i in range(3, len(cometh_chain_R)):
    print(i, cometh_chain_R[i], joint_positions[cometh_chain_R[i]])
    gui.nativeAPI().createSphere(key=cometh_chain_R[i], radii=np.array([0.01,0.01,0.01]), pos=joint_positions[cometh_chain_R[i]], color=np.array([0,0,0,1]))
print()
for i in range(3, len(cometh_chain_L)):
    print(i, cometh_chain_L[i], joint_positions[cometh_chain_L[i]])
    gui.nativeAPI().createSphere(key=cometh_chain_L[i], radii=np.array([0.01,0.01,0.01]), pos=joint_positions[cometh_chain_L[i]], color=np.array([0,0,255,1]))

# skeleton.getJointWorldPositionsMap()
# [print(i, angles[i],round(skeleton.getPositions()[i]*180/np.pi, 2) ) for i in range(0, len(angles))]

In [ ]:
#%% Visualize COMETH and HUB skeletons

def get_angles_HUB(chain):
    HUB_angles = angles_HUB_R if chain==chain_R else angles_HUB_L if chain==chain_L else angles_HUB_H
    return np.array(HUB_angles)

def get_angles_COMETH(chain):
    cometh_dof = cometh_dof_R if chain==chain_R else cometh_dof_L if chain==chain_L else cometh_dof_H
    cometh_angles = np.zeros(len(cometh_dof))
    for i in range(len(cometh_dof)):
        if cometh_dof[i] in angles:
            cometh_angles[i] = skeleton.getPositions()[angles.index(cometh_dof[i])]
    return cometh_angles.reshape(-1, 3)

def get_pos_HUB_FK(skl, chain, Motive=False): 
    bone_length = get_bone_length(skl, chain)
    if Motive:
        eul_angles = get_euler(skl, chain)
    else:
        eul_angles = get_angles_HUB(chain)
    
    fk_pose = np.zeros((len(eul_angles), 3))
    pos_prox = np.zeros(3)
    R_prox = np.eye(3)

    for i in range(2):
        R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='ryxz')[:3, :3]
        R_prox = R_prox @ R_rel
        pos_prox += R_prox @ np.array([0, 0, bone_length[i]])
        fk_pose[i+1] = pos_prox

    for i in range(2, len(eul_angles)-1):
        if chain==chain_R:  
            if i==2:
                R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='ryxz')[:3, :3]
                R_prox = R_prox @ R_rel
                R_off = euler_matrix(0, 0, strnm_ang_R, axes='rzxy')[:3, :3]
                pos_prox += R_prox @ R_off @ np.array([0, 0, bone_length[i]])
            else:
                R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='rzyx')[:3, :3]
                R_prox = R_prox @ R_rel
                pos_prox += R_prox @ np.array([bone_length[i], 0, 0])
        elif chain==chain_L:
            if i==2:
                R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='ryxz')[:3, :3]
                R_prox = R_prox @ R_rel
                R_off = euler_matrix(0, 0, -strnm_ang_L, axes='rzxy')[:3, :3]
                pos_prox += R_prox @ R_off @ np.array([0, 0, bone_length[i]])
            else:
                R_rel = euler_matrix(-eul_angles[i, 0], -eul_angles[i, 1], eul_angles[i, 2], axes='rzyx')[:3, :3]
                R_prox = R_prox @ R_rel
                pos_prox += R_prox @ np.array([-bone_length[i], 0, 0])
        elif chain==chain_H:
            R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='ryxz')[:3, :3]
            R_prox = R_prox @ R_rel
            pos_prox += R_prox @ np.array([0, 0, bone_length[i]])
        fk_pose[i+1] = pos_prox
    return fk_pose

def get_pos_COMETH_FK(chain):
    joint_positions = skeleton.getJointWorldPositionsMap()
    joint_positions_chain = []
    for i in range(len(chain)):
        # rotate to modify x-axis to z-axis, y to -x, z to y
        # R0 = euler_matrix(np.pi/2, np.pi/2, 0, axes='rxyz')[:3, :3]
        # R0 = np.array([[1,0,0], [0,0,1], [0,-1,0]])
        R0 = euler_matrix(0, np.pi/2, np.pi/2, axes='rxyz')[:3, :3] # R0 = [[0,0,1], [1,0,0], [0,1,0]]
        joint_positions_rot = R0 @ joint_positions[chain[i]]
        joint_positions_chain.append(joint_positions_rot)
    joint_positions_chain = np.array(joint_positions_chain) * 1000
    return joint_positions_chain #- joint_positions_chain[0]

# Get bone positions using HUB forward kinematics
pos_fk_R = get_pos_HUB_FK(skl_R, chain_R)
pos_fk_L = get_pos_HUB_FK(skl_L, chain_L)
pos_fk_H = get_pos_HUB_FK(skl_H, chain_H)

# Get bone positions using COMETH forward kinematics
pos_cometh_R = get_pos_COMETH_FK(cometh_chain_R)
pos_cometh_L = get_pos_COMETH_FK(cometh_chain_L)
pos_cometh_H = get_pos_COMETH_FK(cometh_chain_H)

def print_bone_properties(chain, angles=True, positions=True):
    pos_HUB = pos_fk_R if chain==chain_R else pos_fk_L if chain==chain_L else pos_fk_H
    pos_COMETH = pos_cometh_R if chain==chain_R else pos_cometh_L if chain==chain_L else pos_cometh_H
    if angles:
        print("\nCHAIN =        ", chain)
        print("{:<15} {:<25} {:<25}".format("ANGLES", "COMETH", "HUB"))
        for i in range(len(chain)):
            print("{:<15} {:<25} {:<25}".format(chain[i], str(np.around(get_angles_COMETH(chain)[i], 2)), str(np.around(get_angles_HUB(chain)[i], 2))) )
    if positions:
        print("\n{:<15} {:<25} {:<25}".format("POSITIONS", "COMETH", "HUB"))
        for i in range(len(chain)):
            print("{:<15} {:<25} {:<25}".format(chain[i], str(np.around(pos_COMETH[i], 2)), str(np.around(pos_HUB[i], 2))) )
    
print_bone_properties(chain_R)
print_bone_properties(chain_L)

fig, axes = plt.subplots(1, 2, subplot_kw={'projection': '3d'}, figsize=(11, 4.5), constrained_layout=True, num='Joint Locations')
for ax in axes:
    ax.set(xlabel='X', ylabel='Y', zlabel='Z', xlim=(-500, 500), ylim=(-500, 500), zlim=(-100, 900))
    # ax.view_init(elev=25, azim=150)
    ax.view_init(elev=25, azim=160)
    ax.plot([0, 0], [0, 0], [0, 900], 'k--')
    ax.plot([0, 0], [0, 900], [0, 0], 'k--')
    ax.plot([0, 900], [0, 0], [0, 0], 'k--')
ax1, ax2 = axes[0], axes[1]
ax1.set_title('Joint Locations Using COMETH FK', pad=8)
ax2.set_title('Joint Locations Using HUB FK', pad=8)
ax1.set(xlabel='Z', ylabel='X', zlabel='Y', xlim=(-500, 500), ylim=(-500, 500), zlim=(-100, 900))
# ax3.set_title('Joint Locations Using Raw Data', pad=8)
colors_R = ['black', 'orange', 'navy', 'cyan', 'red', 'green', 'blue']
colors_L = ['black', 'orange', 'navy', 'purple', 'magenta', 'lime', 'teal']
for j in range(len(chain_R)):
    ax1.scatter(pos_cometh_R[j][0], pos_cometh_R[j][1], pos_cometh_R[j][2], label=f"{chain_R[j]}, C", marker='*', s=50, color=colors_R[j])
    ax1.scatter(pos_cometh_L[j][0], pos_cometh_L[j][1], pos_cometh_L[j][2], label=f"{chain_L[j]}, C", marker='*', s=50, color=colors_L[j])
    ax2.scatter(pos_fk_R[j][0], pos_fk_R[j][1], pos_fk_R[j][2], label=f"{chain_R[j]}, FK", marker='o', s=50, color=colors_R[j])
    ax2.scatter(pos_fk_L[j][0], pos_fk_L[j][1], pos_fk_L[j][2], label=f"{chain_L[j]}, FK", marker='o', s=50, color=colors_L[j])
    # ax3.scatter(pos_raw_R[j][0], pos_raw_R[j][1], pos_raw_R[j][2], label=f"{chain_R[j]}, Raw", marker='*', s=50, color=colors_R[j])
    # ax3.scatter(pos_raw_L[j][0], pos_raw_L[j][1], pos_raw_L[j][2], label=f"{chain_L[j]}, Raw", marker='*', s=50, color=colors_L[j])
    if j!=0:
        ax1.plot([pos_cometh_R[j][0], pos_cometh_R[j-1][0]], [pos_cometh_R[j][1], pos_cometh_R[j-1][1]], [pos_cometh_R[j][2], pos_cometh_R[j-1][2]], 'k')
        ax1.plot([pos_cometh_L[j][0], pos_cometh_L[j-1][0]], [pos_cometh_L[j][1], pos_cometh_L[j-1][1]], [pos_cometh_L[j][2], pos_cometh_L[j-1][2]], 'b')
        ax2.plot([pos_fk_R[j][0], pos_fk_R[j-1][0]], [pos_fk_R[j][1], pos_fk_R[j-1][1]], [pos_fk_R[j][2], pos_fk_R[j-1][2]], 'k')
        ax2.plot([pos_fk_L[j][0], pos_fk_L[j-1][0]], [pos_fk_L[j][1], pos_fk_L[j-1][1]], [pos_fk_L[j][2], pos_fk_L[j-1][2]], 'b')
        # ax3.plot([pos_raw_R[j][0], pos_raw_R[j-1][0]], [pos_raw_R[j][1], pos_raw_R[j-1][1]], [pos_raw_R[j][2], pos_raw_R[j-1][2]], 'k')
        # ax3.plot([pos_raw_L[j][0], pos_raw_L[j-1][0]], [pos_raw_L[j][1], pos_raw_L[j-1][1]], [pos_raw_L[j][2], pos_raw_L[j-1][2]], 'b')
for j in range(1, len(chain_H)):
    ax1.scatter(pos_cometh_H[j][0], pos_cometh_H[j][1], pos_cometh_H[j][2], label=f"{chain_H[j]}, C", marker='*', s=50, color='black')
    ax2.scatter(pos_fk_H[j][0], pos_fk_H[j][1], pos_fk_H[j][2], label=f"{chain_H[j]}, FK", marker='o', s=50, color='black')
    # ax3.scatter(pos_raw_H[j][0], pos_raw_H[j][1], pos_raw_H[j][2], label=f"{chain_H[j]}, Raw", marker='*', s=50, color='black')
    if j!=0:
        ax1.plot([pos_cometh_H[j][0], pos_cometh_H[j-1][0]], [pos_cometh_H[j][1], pos_cometh_H[j-1][1]], [pos_cometh_H[j][2], pos_cometh_H[j-1][2]], 'k--')
        ax2.plot([pos_fk_H[j][0], pos_fk_H[j-1][0]], [pos_fk_H[j][1], pos_fk_H[j-1][1]], [pos_fk_H[j][2], pos_fk_H[j-1][2]], 'k--')
        # ax3.plot([pos_raw_H[j][0], pos_raw_H[j-1][0]], [pos_raw_H[j][1], pos_raw_H[j-1][1]], [pos_raw_H[j][2], pos_raw_H[j-1][2]], 'k--')
ax2.legend(handles=
    [Line2D([],[], color=colors_R[j], linestyle='', marker='o', markersize=8, label=f"{chain_R[j]}") for j in range(len(chain_R))] + 
    [Line2D([],[], color=colors_L[j], linestyle='', marker='o', markersize=8, label=f"{chain_L[j]}") for j in range(3, len(chain_L))] + 
    [Line2D([],[], color='black', linestyle='-', label='Bones')],  loc='right', bbox_to_anchor=(1.4, 0.5))
# plt.show()
# save figure
fig.savefig('../figures/HUB_COMETH_comparison/joint_locations.png', dpi=300, bbox_inches='tight')

In [ ]:
#%% Visualize COMETH and HUB skeletons at time t

t = 5000
# Offset angle between strnm_dir_R/L and strnm_dir_H
strnm_dir_R = skl_R[('RShoulder', 'Position')].iloc[t] - skl_R[('Chest', 'Position')].iloc[t]
strnm_dir_L = skl_L[('LShoulder', 'Position')].iloc[t] - skl_L[('Chest', 'Position')].iloc[t]
strnm_dir_H = skl_H[('Neck', 'Position')].iloc[t] - skl_H[('Chest', 'Position')].iloc[t]
strnm_ang_R = np.arccos(np.dot(strnm_dir_R, strnm_dir_H)/(np.linalg.norm(strnm_dir_R)*np.linalg.norm(strnm_dir_H)))
strnm_ang_L = np.arccos(np.dot(strnm_dir_L, strnm_dir_H)/(np.linalg.norm(strnm_dir_L)*np.linalg.norm(strnm_dir_H)))

def get_pos_HUB_raw(skl, chain):  
    positions = []
    for j in range(len(chain)):
        pos = skl[(chain[j], 'Position')].iloc[t]
        positions.append(np.array(pos))
    return positions - positions[0]

def get_bone_length(skl, chain):  
    positions = []
    for j in range(len(chain)):
        pos = skl[(chain[j], 'Position')].iloc[t]
        positions.append(np.array(pos))
    positions -= positions[0]
    bone_length = np.linalg.norm(np.diff(positions, axis=0), axis=1)    
    return bone_length

def get_euler(skl, chain):  
    eul = []
    R_prox = np.eye(3)
    for j in range(len(chain)):
        q = skl[(chain[j], 'Rotation')].iloc[t]
        R_global = quaternion_matrix(q)[:3,:3]
        R_rel = R_prox.T @ R_global
        R_prox = R_global
        if chain==chain_H:
            eul_sequnce = 'ryxz'
        else:
            eul_sequnce = 'ryxz' if j < 3 else 'rzyx'
        eul_values = euler_from_matrix(R_rel, axes=eul_sequnce)

        if chain==chain_L and j>3:
            eul_values = [-eul_values[0], -eul_values[1], eul_values[2]]
            
        eul.append(eul_values)
    return np.array(eul)

def get_angles_HUB(chain):
    HUB_angles = angles_HUB_R if chain==chain_R else angles_HUB_L if chain==chain_L else angles_HUB_H
    return np.array(HUB_angles)

def get_angles_COMETH(chain):
    cometh_dof = cometh_dof_R if chain==chain_R else cometh_dof_L if chain==chain_L else cometh_dof_H
    cometh_angles = np.zeros(len(cometh_dof))
    for i in range(len(cometh_dof)):
        if cometh_dof[i] in angles:
            cometh_angles[i] = skeleton.getPositions()[angles.index(cometh_dof[i])]
    return cometh_angles.reshape(-1, 3)

def get_pos_HUB_FK(skl, chain, Motive=False): 
    bone_length = get_bone_length(skl, chain)
    if Motive:
        eul_angles = get_euler(skl, chain)
    else:
        eul_angles = get_angles_HUB(chain)
    
    fk_pose = np.zeros((len(eul_angles), 3))
    pos_prox = np.zeros(3)
    R_prox = np.eye(3)

    for i in range(2):
        R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='ryxz')[:3, :3]
        R_prox = R_prox @ R_rel
        pos_prox += R_prox @ np.array([0, 0, bone_length[i]])
        fk_pose[i+1] = pos_prox

    for i in range(2, len(eul_angles)-1):
        if chain==chain_R:  
            if i==2:
                R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='ryxz')[:3, :3]
                R_prox = R_prox @ R_rel
                R_off = euler_matrix(0, 0, strnm_ang_R, axes='rzxy')[:3, :3]
                pos_prox += R_prox @ R_off @ np.array([0, 0, bone_length[i]])
            else:
                R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='rzyx')[:3, :3]
                R_prox = R_prox @ R_rel
                pos_prox += R_prox @ np.array([bone_length[i], 0, 0])
        elif chain==chain_L:
            if i==2:
                R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='ryxz')[:3, :3]
                R_prox = R_prox @ R_rel
                R_off = euler_matrix(0, 0, -strnm_ang_L, axes='rzxy')[:3, :3]
                pos_prox += R_prox @ R_off @ np.array([0, 0, bone_length[i]])
            else:
                R_rel = euler_matrix(-eul_angles[i, 0], -eul_angles[i, 1], eul_angles[i, 2], axes='rzyx')[:3, :3]
                R_prox = R_prox @ R_rel
                pos_prox += R_prox @ np.array([-bone_length[i], 0, 0])
        elif chain==chain_H:
            R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='ryxz')[:3, :3]
            R_prox = R_prox @ R_rel
            pos_prox += R_prox @ np.array([0, 0, bone_length[i]])
        fk_pose[i+1] = pos_prox
    return fk_pose

def get_pos_COMETH_FK(chain):
    joint_positions = skeleton.getJointWorldPositionsMap()
    joint_positions_chain = []
    for i in range(len(chain)):
        # rotate to modify x-axis to z-axis, y to -x, z to y
        # R0 = euler_matrix(np.pi/2, np.pi/2, 0, axes='rxyz')[:3, :3]
        # R0 = np.array([[1,0,0], [0,0,1], [0,-1,0]])
        R0 = euler_matrix(0, np.pi/2, np.pi/2, axes='rxyz')[:3, :3] # R0 = [[0,0,1], [1,0,0], [0,1,0]]
        joint_positions_rot = R0 @ joint_positions[chain[i]]
        joint_positions_chain.append(joint_positions_rot)
    joint_positions_chain = np.array(joint_positions_chain) * 1000
    return joint_positions_chain #- joint_positions_chain[0]

# # Get bone positions Raw data: from Motive position data
pos_raw_R = get_pos_HUB_raw(skl_R, chain_R)
pos_raw_L = get_pos_HUB_raw(skl_L, chain_L)
pos_raw_H = get_pos_HUB_raw(skl_H, chain_H)

# # Get bone positions using HUB forward kinematics: from Motive quaternions
pos_fk_R = get_pos_HUB_FK(skl_R, chain_R, Motive=True)
pos_fk_L = get_pos_HUB_FK(skl_L, chain_L, Motive=True)
pos_fk_H = get_pos_HUB_FK(skl_H, chain_H, Motive=True)

# Get bone positions using COMETH forward kinematics (at time t in the Motive data)
pos_cometh_R = get_pos_COMETH_FK(cometh_chain_R, Motive=True)
pos_cometh_L = get_pos_COMETH_FK(cometh_chain_L, Motive=True)
pos_cometh_H = get_pos_COMETH_FK(cometh_chain_H, Motive=True)

# Get DOF angles using HUB angle extraction: from Motive quaternions
eul_angles_R = get_euler(skl_R, chain_R)
eul_angles_L = get_euler(skl_L, chain_L)
eul_angles_H = get_euler(skl_H, chain_H)

def print_bone_properties(chain, angles=True, positions=True):
    pos_raw = pos_raw_R if chain==chain_R else pos_raw_L if chain==chain_L else pos_raw_H
    pos_HUB = pos_fk_R if chain==chain_R else pos_fk_L if chain==chain_L else pos_fk_H
    pos_COMETH = pos_cometh_R if chain==chain_R else pos_cometh_L if chain==chain_L else pos_cometh_H
    if angles:
        print("\nCHAIN =        ", chain)
        print("{:<15} {:<25} {:<25}".format("ANGLES", "COMETH", "HUB"))
        for i in range(len(chain)):
            print("{:<15} {:<25} {:<25}".format(chain[i], str(np.around(get_angles_COMETH(chain)[i], 2)), str(np.around(get_angles_HUB(chain)[i], 2))) )
    if positions:
        print("\n{:<15} {:<25} {:<25} {:<25}".format("POSITIONS", "HUB Raw", "HUB FK", "COMETH FK"))
        for i in range(len(chain)):
            print("{:<15} {:<25} {:<25} {:<25}".format(chain[i], str(np.around(pos_raw[i], 2)), str(np.around(pos_HUB[i], 2)), str(np.around(pos_COMETH[i], 2))) )
    
print_bone_properties(chain_R)
print_bone_properties(chain_L)

# print("{:<10} {:<25} {:<25} {:<25}".format("pos_R:", "Raw", "FK", "Euler Angles"))
# for j in range(len(chain_R)):
#     print("{:<10} {:<25} {:<25} {:<25}".format(chain_R[j], str(np.around(pos_raw_R[j], 2)), str(np.around(pos_fk_R[j], 2)), str(np.around(eul_angles_R[j]*180/np.pi, 2))))
# print("{:<10} {:<25} {:<25} {:<25}".format("pos_L:", "Raw", "FK", "Euler Angles"))
# for j in range(len(chain_L)):
#     print("{:<10} {:<25} {:<25} {:<25}".format(chain_L[j], str(np.around(pos_raw_L[j], 2)), str(np.around(pos_fk_L[j], 2)), str(np.around(eul_angles_L[j]*180/np.pi, 2))))        
# for j in range(len(chain_H)):
#     print("{:<10} {:<25} {:<25} {:<25}".format(chain_H[j], str(np.around(pos_raw_H[j], 2)), str(np.around(pos_fk_H[j], 2)), str(np.around(eul_angles_H[j]*180/np.pi, 2))))

colors_R = ['black', 'orange', 'navy', 'cyan', 'red', 'green', 'blue']
colors_L = ['black', 'orange', 'navy', 'purple', 'magenta', 'lime', 'teal']
fig, axes = plt.subplots(1, 3, subplot_kw={'projection': '3d'}, figsize=(16, 4.5), constrained_layout=True, num='Joint Locations')
for ax in axes:
    ax.set(xlabel='X', ylabel='Y', zlabel='Z', xlim=(-500, 500), ylim=(-500, 500), zlim=(-100, 900))
    # ax.view_init(elev=25, azim=150)
    ax.view_init(elev=25, azim=160)
    ax.plot([0, 0], [0, 0], [0, 900], 'k--')
    ax.plot([0, 0], [0, 900], [0, 0], 'k--')
    ax.plot([0, 900], [0, 0], [0, 0], 'k--')
axes[2].set(xlabel='Z', ylabel='X', zlabel='Y', xlim=(-500, 500), ylim=(-500, 500), zlim=(-100, 900))

axes[0].set_title('Bone Locations Raw: using Motive positions', pad=8)
axes[1].set_title('Bone Locations HUB FK: using Motive quats ', pad=8)
axes[2].set_title('Bone Locations COMETH FK: using COMETH FK', pad=8)

for j in range(len(chain_R)):
    axes[0].scatter(pos_raw_R[j][0], pos_raw_R[j][1], pos_raw_R[j][2], label=f"{chain_R[j]}, Raw", marker='*', s=50, color=colors_R[j])
    axes[0].scatter(pos_raw_L[j][0], pos_raw_L[j][1], pos_raw_L[j][2], label=f"{chain_L[j]}, Raw", marker='*', s=50, color=colors_L[j])
    axes[1].scatter(pos_fk_R[j][0], pos_fk_R[j][1], pos_fk_R[j][2], label=f"{chain_R[j]}, FK", marker='o', s=50, color=colors_R[j])
    axes[1].scatter(pos_fk_L[j][0], pos_fk_L[j][1], pos_fk_L[j][2], label=f"{chain_L[j]}, FK", marker='o', s=50, color=colors_L[j])
    axes[2].scatter(pos_cometh_R[j][0], pos_cometh_R[j][1], pos_cometh_R[j][2], label=f"{chain_R[j]}, C", marker='o', s=50, color=colors_R[j])
    axes[2].scatter(pos_cometh_L[j][0], pos_cometh_L[j][1], pos_cometh_L[j][2], label=f"{chain_L[j]}, C", marker='o', s=50, color=colors_L[j])
    if j!=0:
        axes[0].plot([pos_raw_R[j][0], pos_raw_R[j-1][0]], [pos_raw_R[j][1], pos_raw_R[j-1][1]], [pos_raw_R[j][2], pos_raw_R[j-1][2]], 'k')
        axes[0].plot([pos_raw_L[j][0], pos_raw_L[j-1][0]], [pos_raw_L[j][1], pos_raw_L[j-1][1]], [pos_raw_L[j][2], pos_raw_L[j-1][2]], 'b')
        axes[1].plot([pos_fk_R[j][0], pos_fk_R[j-1][0]], [pos_fk_R[j][1], pos_fk_R[j-1][1]], [pos_fk_R[j][2], pos_fk_R[j-1][2]], 'k')
        axes[1].plot([pos_fk_L[j][0], pos_fk_L[j-1][0]], [pos_fk_L[j][1], pos_fk_L[j-1][1]], [pos_fk_L[j][2], pos_fk_L[j-1][2]], 'b')
        axes[2].plot([pos_cometh_R[j][0], pos_cometh_R[j-1][0]], [pos_cometh_R[j][1], pos_cometh_R[j-1][1]], [pos_cometh_R[j][2], pos_cometh_R[j-1][2]], 'k')
        axes[2].plot([pos_cometh_L[j][0], pos_cometh_L[j-1][0]], [pos_cometh_L[j][1], pos_cometh_L[j-1][1]], [pos_cometh_L[j][2], pos_cometh_L[j-1][2]], 'b')
for j in range(1, len(chain_H)):
    axes[0].scatter(pos_raw_H[j][0], pos_raw_H[j][1], pos_raw_H[j][2], label=f"{chain_H[j]}, Raw", marker='*', s=50, color='black')
    axes[1].scatter(pos_fk_H[j][0], pos_fk_H[j][1], pos_fk_H[j][2], label=f"{chain_H[j]}, FK", marker='o', s=50, color='black')
    axes[2].scatter(pos_cometh_H[j][0], pos_cometh_H[j][1], pos_cometh_H[j][2], label=f"{chain_H[j]}, C", marker='o', s=50, color='black')
    if j!=0:
        axes[0].plot([pos_raw_H[j][0], pos_raw_H[j-1][0]], [pos_raw_H[j][1], pos_raw_H[j-1][1]], [pos_raw_H[j][2], pos_raw_H[j-1][2]], 'k--')
        axes[1].plot([pos_fk_H[j][0], pos_fk_H[j-1][0]], [pos_fk_H[j][1], pos_fk_H[j-1][1]], [pos_fk_H[j][2], pos_fk_H[j-1][2]], 'k--')
        axes[2].plot([pos_cometh_H[j][0], pos_cometh_H[j-1][0]], [pos_cometh_H[j][1], pos_cometh_H[j-1][1]], [pos_cometh_H[j][2], pos_cometh_H[j-1][2]], 'k--')

axes[2].legend(handles=
    [Line2D([],[], color=colors_R[j], linestyle='', marker='o', markersize=8, label=f"{chain_R[j]}") for j in range(len(chain_R))] + 
    [Line2D([],[], color=colors_L[j], linestyle='', marker='o', markersize=8, label=f"{chain_L[j]}") for j in range(3, len(chain_L))] + 
    [Line2D([],[], color='black', linestyle='-', label='Bones')],  loc='right', bbox_to_anchor=(1.4, 0.5))

# save figure
fig.savefig('../figures/HUB_COMETH_comparison/bone_locations.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
#%% Visualization of joint locations at time t
def get_euler(skl, chain):  
    eul = []
    R_prox = np.eye(3)
    for j in range(len(chain)):
        q = skl[(chain[j], 'Rotation')].iloc[t]
        R_global = quaternion_matrix(q)[:3,:3]
        R_rel = R_prox.T @ R_global
        R_prox = R_global
        if chain==chain_H:
            eul_sequnce = 'ryxz'
        else:
            eul_sequnce = 'ryxz' if j < 3 else 'rzyx'
        eul_values = euler_from_matrix(R_rel, axes=eul_sequnce)

        if chain==chain_L and j>3:
            eul_values = [-eul_values[0], -eul_values[1], eul_values[2]]
            
        eul.append(eul_values)
    return np.array(eul)

# Get bone positions from raw data
pos_raw_R = get_positions(skl_R, chain_R)
pos_raw_L = get_positions(skl_L, chain_L)
pos_raw_H = get_positions(skl_H, chain_H)

# # Get bone positions from Motive raw data
pos_fk_R = get_pos_HUB_FK(skl_R, chain_R, Motive=True)
pos_fk_L = get_pos_HUB_FK(skl_L, chain_L, Motive=True)
pos_fk_H = get_pos_HUB_FK(skl_H, chain_H, Motive=True)

eul_angles_R = get_euler(skl_R, chain_R)
eul_angles_L = get_euler(skl_L, chain_L)
eul_angles_H = get_euler(skl_H, chain_H)

print("{:<10} {:<25} {:<25} {:<25}".format("pos_R:", "Raw", "FK", "Euler Angles"))
for j in range(len(chain_R)):
    print("{:<10} {:<25} {:<25} {:<25}".format(chain_R[j], str(np.around(pos_raw_R[j], 2)), str(np.around(pos_fk_R[j], 2)), str(np.around(eul_angles_R[j]*180/np.pi, 2))))
print("{:<10} {:<25} {:<25} {:<25}".format("pos_L:", "Raw", "FK", "Euler Angles"))
for j in range(len(chain_L)):
    print("{:<10} {:<25} {:<25} {:<25}".format(chain_L[j], str(np.around(pos_raw_L[j], 2)), str(np.around(pos_fk_L[j], 2)), str(np.around(eul_angles_L[j]*180/np.pi, 2))))        
for j in range(len(chain_H)):
    print("{:<10} {:<25} {:<25} {:<25}".format(chain_H[j], str(np.around(pos_raw_H[j], 2)), str(np.around(pos_fk_H[j], 2)), str(np.around(eul_angles_H[j]*180/np.pi, 2))))

# Plot positions using raw position data and forward kinematics
fig, axes = plt.subplots(1, 2, subplot_kw={'projection': '3d'}, figsize=(11, 4.5), constrained_layout=False, num='Joint Locations')
# fig.subplots_adjust(left=-0.5, wspace=0.08, right=1.2)
for ax in axes:
    ax.set(xlabel='X', ylabel='Y', zlabel='Z', xlim=(-500, 500), ylim=(-500, 500), zlim=(-100, 900))
    # ax.view_init(elev=25, azim=150)
    ax.view_init(elev=25, azim=120)
ax1, ax2 = axes[0], axes[1]
ax1.set_title('Joint Locations Using Position Data', pad=8)
ax2.set_title('Joint Locations Using Forward Kinematics', pad=8)
# colors = ['k', 'y', 'm', 'c', 'r', 'g', 'b']
colors_R = ['black', 'orange', 'navy', 'cyan', 'red', 'green', 'blue']
colors_L = ['black', 'orange', 'navy', 'purple', 'magenta', 'lime', 'teal']
for j in range(len(chain_R)):
    ax1.scatter(pos_raw_R[j][0], pos_raw_R[j][1], pos_raw_R[j][2], label=f"{chain_R[j]}, Raw", marker='*', s=50, color=colors_R[j])
    ax1.scatter(pos_raw_L[j][0], pos_raw_L[j][1], pos_raw_L[j][2], label=f"{chain_L[j]}, Raw", marker='*', s=50, color=colors_L[j])
    ax2.scatter(pos_fk_R[j][0], pos_fk_R[j][1], pos_fk_R[j][2], label=f"{chain_R[j]}, FK", marker='o', s=50, color=colors_R[j])
    ax2.scatter(pos_fk_L[j][0], pos_fk_L[j][1], pos_fk_L[j][2], label=f"{chain_L[j]}, FK", marker='o', s=50, color=colors_L[j])
    if j!=0:
        ax1.plot([pos_raw_R[j][0], pos_raw_R[j-1][0]], [pos_raw_R[j][1], pos_raw_R[j-1][1]], [pos_raw_R[j][2], pos_raw_R[j-1][2]], 'k')
        ax1.plot([pos_raw_L[j][0], pos_raw_L[j-1][0]], [pos_raw_L[j][1], pos_raw_L[j-1][1]], [pos_raw_L[j][2], pos_raw_L[j-1][2]], 'b')
        ax2.plot([pos_fk_R[j][0], pos_fk_R[j-1][0]], [pos_fk_R[j][1], pos_fk_R[j-1][1]], [pos_fk_R[j][2], pos_fk_R[j-1][2]], 'k')
        ax2.plot([pos_fk_L[j][0], pos_fk_L[j-1][0]], [pos_fk_L[j][1], pos_fk_L[j-1][1]], [pos_fk_L[j][2], pos_fk_L[j-1][2]], 'b')
for j in range(1, len(chain_H)):
    ax1.scatter(pos_raw_H[j][0], pos_raw_H[j][1], pos_raw_H[j][2], label=f"{chain_H[j]}, Raw", marker='*', s=50, color='black')
    ax2.scatter(pos_fk_H[j][0], pos_fk_H[j][1], pos_fk_H[j][2], label=f"{chain_H[j]}, FK", marker='o', s=50, color='black')
    if j!=0:
        ax1.plot([pos_raw_H[j][0], pos_raw_H[j-1][0]], [pos_raw_H[j][1], pos_raw_H[j-1][1]], [pos_raw_H[j][2], pos_raw_H[j-1][2]], 'k--')
        ax2.plot([pos_fk_H[j][0], pos_fk_H[j-1][0]], [pos_fk_H[j][1], pos_fk_H[j-1][1]], [pos_fk_H[j][2], pos_fk_H[j-1][2]], 'k--')
ax2.legend(handles=
    [Line2D([],[], color=colors_R[j], linestyle='', marker='o', markersize=8, label=f"{chain_R[j]}") for j in range(len(chain_R))] + 
    [Line2D([],[], color=colors_L[j], linestyle='', marker='o', markersize=8, label=f"{chain_L[j]}") for j in range(3, len(chain_L))] + 
    [Line2D([],[], color='black', linestyle='-', label='Bones')],  loc='right', bbox_to_anchor=(1.4, 0.5))
plt.show()

In [ ]:
# #%% Visualization of joint locations at time t
# t = 1000
# # eul_seq = 'rzxy'
# # eul_seq = 'rxyz'
# eul_seq = 'ryxz'

# # Offset angle between strnm_dir_R/L and strnm_dir_H
# strnm_dir_R = skl_R[('RShoulder', 'Position')] - skl_R[('Chest', 'Position')]
# strnm_dir_L = skl_L[('LShoulder', 'Position')] - skl_L[('Chest', 'Position')]
# strnm_dir_H = skl_H[('Neck', 'Position')] - skl_H[('Chest', 'Position')]
# strnm_ang_R = np.arccos(np.dot(strnm_dir_R.iloc[t], strnm_dir_H.iloc[t])/(np.linalg.norm(strnm_dir_R.iloc[t])*np.linalg.norm(strnm_dir_H.iloc[t])))
# strnm_ang_L = np.arccos(np.dot(strnm_dir_L.iloc[t], strnm_dir_H.iloc[t])/(np.linalg.norm(strnm_dir_L.iloc[t])*np.linalg.norm(strnm_dir_H.iloc[t])))

# def get_positions(skl, chain):  
#     positions = []
#     for j in range(len(chain)):
#         pos = skl[(chain[j], 'Position')].iloc[t]
#         positions.append(np.array(pos))
#     return positions - positions[0]

# def get_euler(skl, chain):  
#     eul = []
#     R_prox = np.eye(3)
#     for j in range(len(chain)):
#         q = skl[(chain[j], 'Rotation')].iloc[t]
#         R_global = quaternion_matrix(q)[:3,:3]
#         R_rel = R_prox.T @ R_global
#         R_prox = R_global
#         if j < 3:
#             eul_values = euler_from_matrix(R_rel, axes='ryxz')
#         else:
#             eul_values = euler_from_matrix(R_rel, axes='rzyx')
#         # (ang1, ang2, ang3) = euler_from_matrix(R_rel, axes=eul_seq)
#         # eul_values = np.array([ang1, ang2, ang3])
#         eul.append(eul_values)
#     return np.array(eul)

# # def forward_kinematics(skl, chain): 
# #     bone_pos = get_positions(skl, chain)
# #     bone_length = np.linalg.norm(np.diff(bone_pos, axis=0), axis=1)
# #     eul_angles = get_euler(skl, chain)
    
# #     fk_pose = np.zeros((len(eul_angles), 3))
# #     pos_prox = np.zeros(3)
# #     R_prox = np.eye(3)
# #     fk_pose[0] = bone_pos[0]
# #     for i in range(2):
# #         R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='rxyz')[:3, :3]
# #         R_prox = R_prox @ R_rel
# #         pos_prox += R_prox @ np.array([0, 0, bone_length[i]])
# #         # pos_prox += R_prox @ np.array([bone_length[i], 0, 0])
# #         # pos_prox += R_prox @ np.array([0, bone_length[i], 0])
# #         fk_pose[i+1] = pos_prox

# #     for i in range(2, len(eul_angles)-1):
# #         R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes=eul_seq)[:3, :3]
# #         R_prox = R_prox @ R_rel
# #         if chain==chain_R:  
# #             if i==2:
# #                 R_off = euler_matrix(0, 0, strnm_ang_R, axes=eul_seq)[:3, :3]
# #                 pos_prox += R_prox @ R_off @ np.array([0, 0, bone_length[i]])
# #             else:
# #                 pos_prox += R_prox @ np.array([bone_length[i], 0, 0])
# #         elif chain==chain_L:
# #             if i==2:
# #                 R_off = euler_matrix(0, 0, -strnm_ang_L, axes=eul_seq)[:3, :3]
# #                 pos_prox += R_prox @ R_off @ np.array([0, 0, bone_length[i]])
# #             else:
# #                 pos_prox += R_prox @ np.array([-bone_length[i], 0, 0])
# #         elif chain==chain_H:
# #             pos_prox += R_prox @ np.array([0, 0, bone_length[i]])
# #         fk_pose[i+1] = pos_prox
# #     return fk_pose

# def forward_kinematics(skl, chain): 
#     bone_pos = get_positions(skl, chain)
#     bone_length = np.linalg.norm(np.diff(bone_pos, axis=0), axis=1)
#     eul_angles = get_euler(skl, chain)

#     fk_pose = np.zeros((len(eul_angles), 3))
#     pos_prox = np.zeros(3)
#     R_prox = np.eye(3)
#     fk_pose[0] = bone_pos[0]
#     for i in range(2):
#         R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='ryxz')[:3, :3]
#         R_prox = R_prox @ R_rel
#         pos_prox += R_prox @ np.array([0, 0, bone_length[i]])
#         # pos_prox += R_prox @ np.array([bone_length[i], 0, 0])
#         # pos_prox += R_prox @ np.array([0, bone_length[i], 0])
#         fk_pose[i+1] = pos_prox

#     for i in range(2, len(eul_angles)-1):
#         if chain==chain_R:  
#             if i==2:
#                 R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes=eul_seq)[:3, :3]
#                 R_prox = R_prox @ R_rel
#                 R_off = euler_matrix(0, 0, strnm_ang_R, axes='rzxy')[:3, :3]
#                 # R_off = np.eye(3)
#                 pos_prox += R_prox @ R_off @ np.array([0, 0, bone_length[i]])
#                 # pos_prox += R_prox @ np.array([bone_length[i], 0, 0])
#             else:
#                 R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='rzyx')[:3, :3]
#                 R_prox = R_prox @ R_rel
#                 pos_prox += R_prox @ np.array([bone_length[i], 0, 0])
#         elif chain==chain_L:
#             if i==2:
#                 R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes=eul_seq)[:3, :3]
#                 R_prox = R_prox @ R_rel
#                 R_off = euler_matrix(0, 0, -strnm_ang_L, axes='rzxy')[:3, :3]
#                 pos_prox += R_prox @ R_off @ np.array([0, 0, bone_length[i]])
#             else:
#                 R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes='rzyx')[:3, :3]
#                 R_prox = R_prox @ R_rel
#                 pos_prox += R_prox @ np.array([-bone_length[i], 0, 0])
#         elif chain==chain_H:
#             R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes=eul_seq)[:3, :3]
#             R_prox = R_prox @ R_rel
#             pos_prox += R_prox @ np.array([0, 0, bone_length[i]])
#         fk_pose[i+1] = pos_prox

#         # for i in range(len(eul_angles)):
#         #     fk_pose[i] = R0.T @ fk_pose[i]
#     return fk_pose

# # Get bone positions from raw data
# pos_raw_R = get_positions(skl_R, chain_R)
# pos_raw_L = get_positions(skl_L, chain_L)
# pos_raw_H = get_positions(skl_H, chain_H)

# # # Get bone positions from Motive raw data
# pos_fk_R = get_pos_HUB_FK(skl_R, chain_R, Motive=True)
# pos_fk_L = get_pos_HUB_FK(skl_L, chain_L, Motive=True)
# pos_fk_H = get_pos_HUB_FK(skl_H, chain_H, Motive=True)

# # # Get bone positions using forward kinematics
# # pos_fk_R = forward_kinematics(skl_R, chain_R)
# # pos_fk_L = forward_kinematics(skl_L, chain_L)
# # pos_fk_H = forward_kinematics(skl_H, chain_H)

# eul_angles_R = get_euler(skl_R, chain_R)
# eul_angles_L = get_euler(skl_L, chain_L)
# eul_angles_H = get_euler(skl_H, chain_H)

# print("{:<10} {:<25} {:<25} {:<25}".format("pos_R:", "Raw", "FK", "Euler Angles"))
# for j in range(len(chain_R)):
#     print("{:<10} {:<25} {:<25} {:<25}".format(chain_R[j], str(np.around(pos_raw_R[j], 2)), str(np.around(pos_fk_R[j], 2)), str(np.around(eul_angles_R[j]*180/np.pi, 2))))
# print("{:<10} {:<25} {:<25} {:<25}".format("pos_L:", "Raw", "FK", "Euler Angles"))
# for j in range(len(chain_L)):
#     print("{:<10} {:<25} {:<25} {:<25}".format(chain_L[j], str(np.around(pos_raw_L[j], 2)), str(np.around(pos_fk_L[j], 2)), str(np.around(eul_angles_L[j]*180/np.pi, 2))))        
# for j in range(len(chain_H)):
#     print("{:<10} {:<25} {:<25} {:<25}".format(chain_H[j], str(np.around(pos_raw_H[j], 2)), str(np.around(pos_fk_H[j], 2)), str(np.around(eul_angles_H[j]*180/np.pi, 2))))

# # Plot positions using raw position data and forward kinematics
# fig, axes = plt.subplots(1, 2, subplot_kw={'projection': '3d'}, figsize=(11, 4.5), constrained_layout=False, num='Joint Locations')
# # fig.subplots_adjust(left=-0.5, wspace=0.08, right=1.2)
# for ax in axes:
#     ax.set(xlabel='X', ylabel='Y', zlabel='Z', xlim=(-500, 500), ylim=(-500, 500), zlim=(-100, 900))
#     # ax.view_init(elev=25, azim=150)
#     ax.view_init(elev=25, azim=120)
# ax1, ax2 = axes[0], axes[1]
# ax1.set_title('Joint Locations Using Position Data', pad=8)
# ax2.set_title('Joint Locations Using Forward Kinematics', pad=8)
# # colors = ['k', 'y', 'm', 'c', 'r', 'g', 'b']
# colors_R = ['black', 'orange', 'navy', 'cyan', 'red', 'green', 'blue']
# colors_L = ['black', 'orange', 'navy', 'purple', 'magenta', 'lime', 'teal']
# for j in range(len(chain_R)):
#     ax1.scatter(pos_raw_R[j][0], pos_raw_R[j][1], pos_raw_R[j][2], label=f"{chain_R[j]}, Raw", marker='*', s=50, color=colors_R[j])
#     ax1.scatter(pos_raw_L[j][0], pos_raw_L[j][1], pos_raw_L[j][2], label=f"{chain_L[j]}, Raw", marker='*', s=50, color=colors_L[j])
#     ax2.scatter(pos_fk_R[j][0], pos_fk_R[j][1], pos_fk_R[j][2], label=f"{chain_R[j]}, FK", marker='o', s=50, color=colors_R[j])
#     ax2.scatter(pos_fk_L[j][0], pos_fk_L[j][1], pos_fk_L[j][2], label=f"{chain_L[j]}, FK", marker='o', s=50, color=colors_L[j])
#     if j!=0:
#         ax1.plot([pos_raw_R[j][0], pos_raw_R[j-1][0]], [pos_raw_R[j][1], pos_raw_R[j-1][1]], [pos_raw_R[j][2], pos_raw_R[j-1][2]], 'k')
#         ax1.plot([pos_raw_L[j][0], pos_raw_L[j-1][0]], [pos_raw_L[j][1], pos_raw_L[j-1][1]], [pos_raw_L[j][2], pos_raw_L[j-1][2]], 'b')
#         ax2.plot([pos_fk_R[j][0], pos_fk_R[j-1][0]], [pos_fk_R[j][1], pos_fk_R[j-1][1]], [pos_fk_R[j][2], pos_fk_R[j-1][2]], 'k')
#         ax2.plot([pos_fk_L[j][0], pos_fk_L[j-1][0]], [pos_fk_L[j][1], pos_fk_L[j-1][1]], [pos_fk_L[j][2], pos_fk_L[j-1][2]], 'b')
# for j in range(1, len(chain_H)):
#     ax1.scatter(pos_raw_H[j][0], pos_raw_H[j][1], pos_raw_H[j][2], label=f"{chain_H[j]}, Raw", marker='*', s=50, color='black')
#     ax2.scatter(pos_fk_H[j][0], pos_fk_H[j][1], pos_fk_H[j][2], label=f"{chain_H[j]}, FK", marker='o', s=50, color='black')
#     if j!=0:
#         ax1.plot([pos_raw_H[j][0], pos_raw_H[j-1][0]], [pos_raw_H[j][1], pos_raw_H[j-1][1]], [pos_raw_H[j][2], pos_raw_H[j-1][2]], 'k--')
#         ax2.plot([pos_fk_H[j][0], pos_fk_H[j-1][0]], [pos_fk_H[j][1], pos_fk_H[j-1][1]], [pos_fk_H[j][2], pos_fk_H[j-1][2]], 'k--')
# ax2.legend(handles=
#     [Line2D([],[], color=colors_R[j], linestyle='', marker='o', markersize=8, label=f"{chain_R[j]}") for j in range(len(chain_R))] + 
#     [Line2D([],[], color=colors_L[j], linestyle='', marker='o', markersize=8, label=f"{chain_L[j]}") for j in range(3, len(chain_L))] + 
#     [Line2D([],[], color='black', linestyle='-', label='Bones')],  loc='right', bbox_to_anchor=(1.4, 0.5))
# plt.show()